<a href="https://colab.research.google.com/github/jorden17/Prediction_Sysytem/blob/main/Heart_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")


In [41]:
df= pd.read_csv('/content/heart.csv')

In [43]:
X = df.drop(columns='target', axis=1)
y = df['target']

In [55]:
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
# Step 5: Scale the Data (RF doesn't need it, but we’ll scale for consistency)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [66]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [68]:
# 3. Model Selection and Training (Random Forest)
# Random Forest is often a good choice for this type of problem.

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [69]:
# 4. Model Evaluation
y_pred = best_rf.predict(X_test_scaled)

print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
Accuracy: 0.8524590163934426
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.85        29
           1       0.87      0.84      0.86        32

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61

Confusion Matrix:
 [[25  4]
 [ 5 27]]


In [71]:

# 5. Prediction with new data.
def predict_heart_disease(input_data, scaler, model):
    """Predicts heart disease using the trained model."""
    input_df = pd.DataFrame([input_data], columns=X.columns)
    input_scaled = scaler.transform(input_df)
    prediction = model.predict(input_scaled)
    return "Heart Disease" if prediction[0] == 1 else "Healthy"


In [74]:

# Example usage of the prediction function.
new_input = (48, 1, 2, 110, 229, 0, 0, 168, 0, 1.0, 1, 0, 3)#Example Input from previous conversation.
prediction = predict_heart_disease(new_input, scaler, best_rf)
print(f"Prediction for new input: {prediction}")


Prediction for new input: Heart Disease


In [77]:
# if you want to save your model.
import pickle
pickle.dump(best_rf, open('heart_model.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))


In [78]:
# to load the model.
loaded_model = pickle.load(open('heart_disease_model.pkl', 'rb'))
loaded_scaler = pickle.load(open('scaler.pkl', 'rb'))